In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import numpy as np
import pandas as pd

In [ ]:
# download dataset from GDrive
downloaded1 = drive.CreateFile({'id':'1Covd9-Ex62b29BM3_IEdZR8TQtfJB4Ie'})
downloaded2 = drive.CreateFile({'id':'1rqO6YiJncEfRe4jdXKabB_NdsKnSUH1V'})
downloaded3 = drive.CreateFile({'id':'1dOli08qO5MgRfOyjO0myRNK5NOqTgJVy'})
downloaded4 = drive.CreateFile({'id':'1AIYPhOnn_eN1Y8QoHU8Ik0N1yFPVGcJ9'})

downloaded1.GetContentFile('dataset1.csv')
downloaded2.GetContentFile('dataset2.csv')
downloaded3.GetContentFile('dataset3.csv')
downloaded4.GetContentFile('dataset4.csv')

# Read file as panda dataframe
dfRaw = pd.read_csv('dataset1.csv')
dfRaw_Stemmed = pd.read_csv('dataset2.csv')
dfPre = pd.read_csv('dataset3.csv')
dfPre_Stemmed = pd.read_csv('dataset4.csv')

In [ ]:
print("Banyak dfRaw         : ", len(dfRaw))
print("Banyak dfRaw_Stemmed : ", len(dfRaw_Stemmed))
print("Banyak dfPre         : ", len(dfPre))
print("Banyak dfPre_Stemmed : ", len(dfPre_Stemmed))

Banyak dfRaw         :  2680
Banyak dfRaw_Stemmed :  2680
Banyak dfPre         :  2680
Banyak dfPre_Stemmed :  2680


# Split each dataset into train and test

In [ ]:
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
import math

In [ ]:
# Split dfRaw
X = dfRaw['tweet'].values
y = dfRaw['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print("dfRaw")
print("Banyak data train : ", len(X_train))
print("Banyak data test : ", len(X_test))
total_raw = len(X_train) + len(X_test)
print("Total : " , total_raw, ". Apakah sesuai ?", total_raw == len(dfRaw))

dfRaw
Banyak data train :  1876
Banyak data test :  804
Total :  2680 . Apakah sesuai ? True


In [ ]:
dfTrain_Raw = dfRaw[dfRaw.tweet.isin(X_train)]
dfTrain_Raw

,tweet,label
0,Peneliti: Presiden perlu bentuk Badan Otorita ...,0
1,Pembangunan PLTN di Kalbar mantapkan pemindaha...,1
2,Prabowo setuju karena pemindahan ibu kota suda...,1
3,Pemindahan ibu kota bukti Jokowi ingin adanya ...,1
4,Lah itu kan sdh mau di ringankan dg pemindahan...,1
...,...,...
2672,"Cocok dah pemindahan ibu ota, biar aja buaya k...",1
2673,Apakah pemindahan ibu kota negara ke Kalimanta...,0
2674,Amien Rais: Pemindahan ibu kota mempercepat pe...,-1
2677,Pemindahan ibu kota berhubungan dengan CINA Ke...,0


In [ ]:
dfTest_Raw = dfRaw[dfRaw.tweet.isin(X_test)]
dfTest_Raw

,tweet,label
18,Aspek historis calon ibu kota negara Kutai Ka...,0
25,Gubernur Kaltim: Pemindahan Ibu Kota di Bukit ...,0
29,Politikus Demokrat sebut Pemindahan Ibu Kota H...,-1
30,"Lagi pula, pemindahan ibu kota ke Kaltim terke...",-1
32,Negara dikelola secara sporadis.. #pemindahani...,0
...,...,...
2669,Fadli Zon Pertanyakan Urgensi Pemindahan Ibu K...,0
2671,Pemindahan Ibu Kota dengan skema Investasi dar...,1
2675,Semoga hal serupa juga dilakukan saat proses p...,1
2676,Tolak Pemindahan Ibu Kota Tolak Kenaikan Iuran...,-1


In [ ]:
dfTrain_RawStem = dfRaw_Stemmed.iloc[dfTrain_Raw.index]
dfTrain_RawStem

,tweet,label
0,teliti presiden perlu bentuk badan otorita pin...,0
1,bangun pltn di kalbar mantap pindah ibu kota n...,1
2,prabowo tuju karena pindah ibu kota sudah jadi...,1
3,pindah ibu kota bukti jokowi ingin ada perata ...,1
4,lah itu kan sdh mau di ringan dg pindah ibu kota,1
...,...,...
2672,cocok dah pindah ibu ota biar aja buaya kota a...,1
2673,apakah pindah ibu kota negara ke kalimantan ti...,0
2674,amien rais pindah ibu kota cepat pengambilalih...,-1
2677,pindah ibu kota hubung dengan cina rusuh di pa...,0


In [ ]:
dfTest_RawStem = dfRaw_Stemmed.iloc[dfTest_Raw.index]
dfTest_RawStem

,tweet,label
18,aspek historis calon ibu kota negara kuta kart...,0
25,gubernur kaltim pindah ibu kota di bukit soeha...,0
29,politikus demokrat sebut pindah ibu kota harus...,-1
30,lagi pula pindah ibu kota ke kaltim kes jadi p...,-1
32,negara kelola cara sporadis pemindahanibukota,0
...,...,...
2669,fadli zon tanya urgensi pindah ibu kota pic tw...,0
2671,pindah ibu kota dengan skema investasi dari ju...,1
2675,moga hal rupa juga laku saat proses pindah ibu...,1
2676,tolak pindah ibu kota tolak naik iur bpjs tola...,-1


In [ ]:
dfTrain_Pre = dfPre.iloc[dfTrain_Raw.index]
dfTrain_Pre

,tweet,label
0,peneliti presiden perlu bentuk badan otorita p...,0
1,pembangunan pltn kalbar mantapkan pemindahan ...,1
2,prabowo setuju pemindahan ibu kota menjadi u...,1
3,pemindahan ibu kota bukti jokowi adanya pemer...,1
4,lah kan sdh mau ringankan dg pemindahan ibu ...,1
...,...,...
2672,cocok dah pemindahan ibu ota biar aja buaya ko...,1
2673,pemindahan ibu kota negara kalimantan timur ...,0
2674,amien rais pemindahan ibu kota mempercepat pen...,-1
2677,pemindahan ibu kota berhubungan cina kerusuha...,0


In [ ]:
dfTest_Pre = dfPre.iloc[dfTest_Raw.index]
dfTest_Pre

,tweet,label
18,aspek historis calon ibu kota negara kutai kar...,0
25,gubernur kaltim pemindahan ibu kota bukit soe...,0
29,politikus demokrat sebut pemindahan ibu kota h...,-1
30,pemindahan ibu kota kaltim terkesan jadi ke...,-1
32,negara dikelola sporadis,0
...,...,...
2669,fadli zon pertanyakan urgensi pemindahan ibu k...,0
2671,pemindahan ibu kota skema investasi jual bel...,1
2675,semoga serupa dilakukan proses pemindahan i...,1
2676,tolak pemindahan ibu kota tolak kenaikan iuran...,-1


In [ ]:
dfTrain_PreStem = dfPre_Stemmed.iloc[dfTrain_Raw.index]
dfTrain_PreStem

,tweet,label
0,teliti presiden perlu bentuk badan otorita pin...,0
1,bangun pltn kalbar mantap pindah ibu kota neg...,1
2,prabowo tuju pindah ibu kota jadi usul renca...,1
3,pindah ibu kota bukti jokowi ada perata bangun,1
4,lah kan sdh mau ringan dg pindah ibu kota,1
...,...,...
2672,cocok dah pindah ibu ota biar aja buaya kota a...,1
2673,pindah ibu kota negara kalimantan timur sol...,0
2674,amien rais pindah ibu kota cepat pengambilalih...,-1
2677,pindah ibu kota hubung cina rusuh papua hubu...,0


In [ ]:
dfTest_PreStem = dfPre_Stemmed.iloc[dfTest_Raw.index]
dfTest_PreStem

,tweet,label
18,aspek historis calon ibu kota negara kuta kart...,0
25,gubernur kaltim pindah ibu kota bukit soehart...,0
29,politikus demokrat sebut pindah ibu kota harus...,-1
30,pindah ibu kota kaltim kes jadi putus final...,-1
32,negara kelola sporadis,0
...,...,...
2669,fadli zon tanya urgensi pindah ibu kota pic tw...,0
2671,pindah ibu kota skema investasi jual beli la...,1
2675,moga rupa laku proses pindah ibu kota pros...,1
2676,tolak pindah ibu kota tolak naik iur bpjs tola...,-1


In [ ]:
dfTrain_PreStem[dfTrain_PreStem['tweet'].isnull() == True]

,tweet,label


In [ ]:
dfTrain_PreStem[dfTrain_PreStem['label'].isnull() == True]

,tweet,label


In [ ]:
dfTest_PreStem[dfTest_PreStem['tweet'].isnull() == True]

,tweet,label


In [ ]:
dfTest_PreStem[dfTest_PreStem['label'].isnull() == True]

,tweet,label


In [ ]:
dfTrain_Pre[dfTrain_Pre['tweet'].isnull() == True]

,tweet,label


In [ ]:
dfTrain_Pre[dfTrain_Pre['label'].isnull() == True]

,tweet,label


# Export

In [ ]:
from google.colab import files

dfTrain_Raw.to_csv('dfTrain_Raw.csv', index = False)
dfTest_Raw.to_csv('dfTest_Raw.csv', index = False)

dfTrain_RawStem.to_csv('dfTrain_RawStem.csv', index = False)
dfTest_RawStem.to_csv('dfTest_RawStem.csv', index = False)

dfTrain_Pre.to_csv('dfTrain_Pre.csv', index = False)
dfTest_Pre.to_csv('dfTest_Pre.csv', index = False)

dfTrain_PreStem.to_csv('dfTrain_PreStem.csv', index = False)
dfTest_PreStem.to_csv('dfTest_PreStem.csv', index = False)

files.download("dfTrain_Raw.csv")
files.download("dfTest_Raw.csv")

files.download("dfTrain_RawStem.csv")
files.download("dfTest_RawStem.csv")

files.download("dfTrain_Pre.csv")
files.download("dfTest_Pre.csv")

files.download("dfTrain_PreStem.csv")
files.download("dfTest_PreStem.csv")